In [32]:
import os
import dspy
import mlflow
import pandas as pd
from dotenv import load_dotenv

In [33]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
lm = dspy.LM("gpt-4o", api_key=openai_api_key)

In [34]:
local_lm = dspy.LM("ollama_chat/llama3.2", api_base="http://localhost:11434", api_key="")
dspy.configure(lm=local_lm)

In [36]:
#mlflow ui --port 5000
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("DSPy")
mlflow.dspy.autolog()

2025/05/17 11:20:17 INFO mlflow.tracking.fluent: Experiment with name 'DSPy' does not exist. Creating a new experiment.


In [ ]:
class Chat(dspy.Signature):
    "You are a helpful assistant."
    question: str = dspy.InputField(desc="Questions asked by the user")
    answer: str = dspy.OutputField(desc="Answer to the question")

class Model(dspy.Module):
    def __init__(self):
        super().__init__()
        self.respond = dspy.ReAct(Chat)

    def forward(self, question: str):
        return self.respond(question=question)

In [ ]:
path = kagglehub.dataset_download("bishnushahi/dog-cat-qa")

print("Path to dataset files:", path)